In [1]:
from datasets import load_dataset

ds = load_dataset("qiaojin/PubMedQA", "pqa_labeled")
ds_art = load_dataset("qiaojin/PubMedQA", "pqa_artificial")

/home/tadesa1/anaconda3/envs/nlp_ml/lib/python3.9/site-packages/tqdm-4.67.1-py3.9.egg/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds['train'].num_rows

1000

In [3]:
import random

window_size = 2500
lower_b = random.randint(0, ds_art['train'].num_rows - window_size)
upper_b = lower_b + window_size

pubmed_qa_list = []
train_ds = ds['train']
train_ds_art = ds_art['train']
pubmed_qa_list = list(train_ds)
pubmed_qa_list.extend(list(train_ds_art)[lower_b:upper_b])

In [4]:
len(pubmed_qa_list)

3500

In [5]:
import os
import json
import glob

def process_json_files(directory_path, output_file):
    # Dictionary to store all samples
    all_samples = {
        "samples": []
    }
    
    # Find all JSON files in the directory
    json_files = glob.glob(os.path.join(directory_path, "*.json"))
    
    # Process each file
    for file_path in json_files:
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
                
                # Check if the file has a "samples" key
                if "samples" in data:
                    # Add samples to our collection
                    all_samples["samples"].extend(data["samples"])
                    print(f"Added {len(data['samples'])} samples from {file_path}")
                else:
                    print(f"Warning: No 'samples' key found in {file_path}")
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
    
    # Save all samples to the output file
    with open(output_file, 'a', encoding='utf-8') as outfile:
        json.dump(all_samples, outfile, indent=2, ensure_ascii=False)
    
    print(f"Successfully saved {len(all_samples['samples'])} samples to {output_file}")


In [6]:
directory_path = "/home/tadesa1/research/ADBMO-UNLV/t5_super_finetune/data/"
output_file = "/home/tadesa1/research/ADBMO-UNLV/t5_super_finetune/data/synth_data_pcls_two.json"

for root, dir, files in os.walk(directory_path):
    for directory in dir:
        process_json_files(f"{directory_path}/{directory}", output_file)
        # print(f"Processed files in {directory}")
        print("--------------------------------")
    break


Added 91 samples from /home/tadesa1/research/ADBMO-UNLV/t5_super_finetune/data//criteria_4/combined_samples_20250429_233800.json
Added 95 samples from /home/tadesa1/research/ADBMO-UNLV/t5_super_finetune/data//criteria_4/combined_samples_20250501_163140.json
Added 90 samples from /home/tadesa1/research/ADBMO-UNLV/t5_super_finetune/data//criteria_4/combined_samples_20250501_163530.json
Added 20 samples from /home/tadesa1/research/ADBMO-UNLV/t5_super_finetune/data//criteria_4/combined_samples_20250501_164031.json
Successfully saved 296 samples to /home/tadesa1/research/ADBMO-UNLV/t5_super_finetune/data/synth_data_pcls_two.json
--------------------------------
Added 99 samples from /home/tadesa1/research/ADBMO-UNLV/t5_super_finetune/data//criteria_3/combined_samples_20250501_160538.json
Added 108 samples from /home/tadesa1/research/ADBMO-UNLV/t5_super_finetune/data//criteria_3/combined_samples_20250501_155226.json
Added 20 samples from /home/tadesa1/research/ADBMO-UNLV/t5_super_finetune/da

In [8]:
with open("/home/tadesa1/research/ADBMO-UNLV/t5_super_finetune/data/synth_data_pcls_two.json", 'r', encoding='utf-8') as file:
    synthetic_data = json.load(file)

examples = []
for sample in synthetic_data.get("samples", []):
    inp = sample["input"]
    out = sample["output"]
    examples.append((inp, out))

In [9]:
len(examples)

3668

In [10]:
for doc in pubmed_qa_list:
    print("".join(doc['context']['contexts']))
    break

Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants.The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), cells in early stages of PCD (EPCD), and cells in late stages of PCD (LPCD). Window stage leaves were stained with the mitochondrial dye MitoTracker Red CMXRos and

In [ ]:
for doc in pubmed_qa_list:
    # build the instruction + context
    context = "".join(doc['context']['contexts'])
    inp = f"Instruction: {doc['question']} Text: {context}"
    # yes/no answer plus the justification (long_answer)
    ans = doc["final_decision"].strip().lower()
    just = doc.get("long_answer", "").strip()
    out = f"Answer: {ans}. {just}"
    examples.append((inp, out))

In [ ]:
examples[5000]

("Instruction: Is ambulatory high-dose methotrexate administration among pediatric osteosarcoma patients in an urban , underserved setting feasible , safe , and cost-effective? Text: We describe the safety, feasibility, and provide a cost-estimate of outpatient high-dose methotrexate administration (HDMTX) among an urban, underserved population.A retrospective analysis of ambulatory HDMTX administration among osteosarcoma patients, at Montefiore Medical Center's Children's Hospital (Bronx, NY) was performed. HDMTX (12 g/m(2)) was given intravenously (IV) over 4 hr after urine alkalinization. Patients were discharged home to continue IV hydration and alkalinization delivered via a home infusion pump. Families were instructed to monitor urine pH overnight and management was adjusted according to our institution's treatment algorithm until MTX level ≤ 0.1\u2009µmol/L. A cost estimate was performed to assess the difference in costs for outpatient versus hypothetical inpatient administratio

In [11]:
from transformers import T5TokenizerFast
from torch.utils.data import DataLoader, DistributedSampler, random_split
from dataset_utils import T5Dataset, DataCollatorForT5
from datasets import load_dataset
import json

tokenizer = T5TokenizerFast.from_pretrained(
    "/home/tadesa1/research/ADBMO-UNLV/t5_lora_finetuning/t5_finetuned_best")

RED = "\033[31m"
GREEN = "\033[32m"
YELLOW = "\033[33m"
BLUE = "\033[34m"
BOLD = "\033[1m"
RESET = "\033[0m"

ds = load_dataset("qiaojin/PubMedQA", "pqa_labeled")
ds_art = load_dataset("qiaojin/PubMedQA", "pqa_artificial")

window_size = 2500
lower_b = random.randint(0, ds_art['train'].num_rows - window_size)
upper_b = lower_b + window_size

pubmed_qa_data = []
train_ds = ds['train']
train_ds_art = ds_art['train']
pubmed_qa_data = list(train_ds)
pubmed_qa_data.extend(list(train_ds_art)[lower_b:upper_b])



print(f"{BOLD}==========================================={GREEN}Received PQA Dataset: {len(pubmed_qa_data)}{RESET}===========================================")

with open("/home/tadesa1/research/ADBMO-UNLV/t5_super_finetune/data/synth_data_pcls_two.json", 'r', encoding='utf-8') as file:
    synthetic_data = json.load(file)



dataset = T5Dataset(
    synthetic_data=synthetic_data,
    pubmed_data=pubmed_qa_data,
    tokenizer=tokenizer,
    max_input_length=1024,
    max_target_length=512
)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

===========================================Received PQA Dataset: 3500===========================================


In [19]:
random.seed(42)
random.shuffle(dataset.examples)

In [21]:
i = 0
for example in dataset.examples:
    print(example)
    if i > 10:
        break
    i += 1

('Instruction: Does use of heparin versus lepirudin flush to prevent withdrawal occlusion of central venous access devices? Text: To determine whether lepirudin flushes are more effective than heparinized saline in preventing withdrawal occlusion of central venous access devices.Randomized, double-blind clinical trial.Research institution-tertiary referral center.Forty-nine adults undergoing bone marrow transplantation for hematologic malignancies or metastatic solid tumors.Twenty-four patients received heparin and 25 received lepirudin flushes. The heparin dose was 3 ml of porcine heparin 100 U/ml (300 U) per catheter lumen at least once/day; the lepirudin dose was 3 ml of lepirudin 100 microg/ml (300 microg) per catheter lumen at least once/day. After 3-4 weeks, all 49 patients received the heparin flushes.Efficacy was assessed by the frequency with which the patients were treated with alteplase instillations for withdrawal occlusion of their central venous access devices during the 

In [13]:
print(len(train_dataset))
print(len(val_dataset))

5734
1434


In [49]:
loader = DataLoader(
  train_dataset,
  batch_size=4,
  shuffle=True,
  collate_fn=DataCollatorForT5(tokenizer),
)
batch = next(iter(loader))
decoded = tokenizer.batch_decode(batch["input_ids"], skip_special_tokens=True)
print("Sample inputs:", decoded[:1200])

Sample inputs: ['Instruction: Are single nucleotide polymorphisms in AGTR1 , TFAP2B , and TRAF1 associated with the incidence of patent ductus arteriosus in Japanese preterm infants? Text: Persistent patent ductus arteriosus (PDA) is a frequent complication in preterm infants. Single nucleotide polymorphisms (SNP) in several genes, including angiotensin II receptor, type 1 (AGTR1), transcription factor AP-2 beta (TFAP2B) and tumor necrosis factor receptor-associated factor 1 (TRAF1), have been reported to be associated with PDA in preterm infants. The aim of this study was to evaluate the relationships between PDA in preterm infants and polymorphisms in AGTR1, TFAP2B and TRAF1 in the Japanese population.The subjects consisted of 107 preterm infants with gestational age 32 weeks. Extremely low-birthweight infants were treated with prophylactic indomethacin during the first 24 h after birth. Five SNP, namely, rs5186 in AGTR1, rs987237 and rs6930924 in TFAP2B, and rs1056567 and rs10985070